### Derive bathymetry and hypsometry from a static topography dataset
This notebook demonstrates the entire process to get a bathymetric dataset for a lake with a clear outflow point such as a dam. The notebook uses two classes to explain the entire process, and uses an example dataset, delivered along with this repository.

Let's start with loading in several subpackages.

In [ ]:
from bathypy.static_hypso import Bathymetry  # this is the class demonstrated in this notebook
from bathypy import gww_requests  # gww_requests provides access to the global water watch database. Used to get a polygon of a reservoir
import xarray as xr  # xarray is the main data crunching library used for the functionalities
from shapely.geometry import shape
import geemap
import geopandas as gpd
import matplotlib.pyplot as plt
import ipyleaflet
import json

### Get a polygon of interes
Now that we have our libraries setup, let's first get a polygon of the lake we are interested in. In our case, we retrieve this
from our Global Water Watch database through the API, using a reservoir id. You can also simply load in a shapefile and extract the geometry. As long as you end up with a shapely.geometry.Polygon or MultiPolygon as object, you are in good shape.

In [ ]:
reservoir_id = 88643 #Mita Hills, Zambia. Or any other ID.
response = gww_requests.get_reservoir(reservoir_id)
polygon = shape(response.json()["geometry"])
polygon


This polygon has a logical location on the globe. Let's plot that



In [ ]:
xy = polygon.centroid.xy[1][0], polygon.centroid.xy[0][0]
xy

In [ ]:
Map = ipyleaflet.Map(center=xy, zoom=9)
# Map = geemap.Map(center=xy, zoom=10)

In [ ]:
gdf = gpd.GeoDataFrame({"id": [reservoir_id]}, geometry=[polygon])
gdf.to_file("Mita_Hills.geojson")
layer = ipyleaflet.GeoJSON(data=json.loads(gdf.to_json()))
Map.add(layer)

### Make a Bathymetry object, starting with the reservoir polygon
This only starts an object with a simple polygon.

In [ ]:
bath = Bathymetry(polygon)

We need to supply a topography dataset with the bathymetry object. If you have a HydroMT data catalog available, you can retrieve this with the method ``get_topography_from_hydromt`` with the first input the .yml file of your data catalog. As keyword ``path_or_key`` you should then supply the name of the dataset containing terrain, flow directions, slopes, and so on. We recommend supplying a buffer of 30 grid cells (``buffer=30``) to ensure you have an area slightly larger than the reservoir polygon. The polygon will be used to find the area of interest. Below a commented example is provided.

In [ ]:
# # get data from hydroMT, and clip the data to the area_fit extend of the reservoir
# data_catalog = "./example_catalog.yml"
# bath.get_topography_from_hydromt(
#     data_catalog,
#     add_flow_direction=False,
#     add_derivatives=False,
#     add_mask=False,
#     path_or_key="merit_hydro",
#     buffer=30
# )


An alternative is that you already have a prepared dataset with topography layers in a file. For this example, we will use a prepared dataset cut out of MERIT-Hydro, prepared using HydroMT. It covers the area around the polygon supplied.

In [ ]:
fn = "datasets/topo_mita.nc"
# open the data with normal xarray functionality
ds = xr.open_dataset(fn)
# add dataset to topography
bath.set_topography(ds)
bath.ds_topography

You can see that the dataset is added as a property in ``ds_topography``. A number of compulsory layers must be present, which all are available in the MERIT-Hydro dataset. The required layers are:

``elevtn``: elevation [m]
``lndslp``: local slope [-]
``flwdir``: 8D flow network in a format compatible with the ``pyflwdir`` package (see https://deltares.github.io/pyflwdir/latest/index.html)
``uparea``: upstream area [m2]
``strord``: strahler stream order [-]

Furthermore some additional layers are added on-the-fly such as ``bathymetry``. Also the ``flwdir`` layer will be translated to a flow intelligent layer automatically when adding the dataset. With normal xarray functionalities, we can plot layers.

In [ ]:
f = plt.figure(figsize=(6, 6))
bath.ds_topography["bathymetry"].plot()

This layer contains the topography data, with the reservoir area filtered out. A nicer plot can be made with the ``plot_topo`` method, which only works if there are no missing values in the field. The original elevation layer of MERIT-Hydro is a good example.

In [ ]:
bath.plot_topo(field="elevtn")

There are several methods to fill missing values in the ``bathymetry`` layer. An obvious one is ``interp`` which simply interpolates all values with a 2D interpolation method. If applied directly this will yield more or less the same result as our original elevation, as interpolation will occur from shore to shore. Hence, we first need elevation values somewhere in the middle of the reservoir. We use the ``skeletonize_bathymetry`` method to establish this. It will create a skeleton of elevation values from the shoreline that contains elevation, to the most downstream point where the missing area stops and elevation values are available. This process can be repeated starting with the largest strahler stream order found, that bounds the missing area, then moving to lower stream orders consecutively. Elevation values are filled using the method by Messager et al. (2016).


$dh(x) = ix ^b$

where $dh$ is the difference in elevation between shore and a point of interest along the stream under consideration, $i$ is the slope at the shore, $x$ the distance from the shoreline and $b$ a power that defines the reduction of the increase in water level difference with distance.

The equation is solved over the stream network using an 8D river network, therefore it is important that a data layer ``flwdir`` is available in the topography dataset. Let's apply the skeletonization for 5 stream orders.


In [ ]:
bath.skeletonize_bathymetry(iter=5)


The bathymetry should now contain more data. Let's use ``xarray`` plotting to visualize what we have. We make the figure larger to ensure we can see the stream pixels.

In [ ]:
f = plt.figure(figsize=(14, 12))
bath.ds_topography["bathymetry"].plot()

You should see that the stream network within the masked areas is now filled with elevation values. Also smaller streams are filled and even streams with yet again smaller stream orders.

### plot transect of stream elevation
There is also a special function that plots the behaviour of the elevation over the main stream. This is useful to investigate if the dam wall has been properly detected. Below we plot both the original elevation and the modified bathymetry in this way.

In [ ]:

ax = plt.axes()
bath.plot_along_stream(ax=ax, field="bathymetry")
bath.plot_along_stream(ax=ax, field="elevtn")


The plot shows that the dam wall was automatically found, but that the bottom profile does not yet properly connect to the original river profile downstream of the dam. We can reposition the location from which we mask out upstream values with a property called ``downstream_reservoir``. By default this is set to zero, which then positions that location as closely downstream of the dam location as possible. We can also set that to 500 meters for instance. Let's do the processing again, but now with a 500 meter downstream displacement.

In [ ]:
bath = Bathymetry(polygon, downstream_reservoir=500)
# set the topo
bath.set_topography(ds)
bath.skeletonize_bathymetry(iter=5)
ax = plt.axes()
bath.plot_along_stream(ax=ax, field="bathymetry")
bath.plot_along_stream(ax=ax, field="elevtn")


This looks more realistic. But at this stage all the in between areas are not yet available. Just calling the ``interpolate`` method fills in the blancs. We can apply a number of extra keywords to enhance shading. These are passed to ``matplotlib.colors.LightSource.shade``. Please look at the documentation to see which keyword arguments you can pass.

In [ ]:
f, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)
bath.interpolate()
bath.plot_topo(
    ax=ax1,
    field="elevtn",
    vert_exag=20,
    blend_mode="soft"
)
bath.plot_topo(
    ax=ax2,
    field="bathymetry",
    vert_exag=20,
    blend_mode="soft"
)

The right-side shows our interpreted bathymetry. It looks like a pretty natural bathymetry. We can now retrieve a hypsometry object from this. Hypsometry describes the geometrical relationships between elevation, surface area and (through integration) volume. A hypsometry object can be created by sampling elevation values and the associated surface area that is inundated according to the bathymetry with these elevation values. A relationship between surface area and elevation can then be fitted, which can be used to predict a geometric variable from another geometric variable.

The hypsometric relationship between surface area and elevation is described by.

$A(h) = a\left(h-h_0\right)^b$

This functionality is encapsulated in the ``Hypsometry`` class. A hypsometry object can be directly derived from a ``Bathymetry`` object with the ``get_hypsometry`` method. You can define the vertical resolution over which to sample, and extend the sampling to above the dam wall if you wish so. 

In [ ]:
hypso = bath.get_hypsometry(add_to_max_h=0., resolution=1.)
hypso

The hypsometry relationship has some convenience plotting functions, with which we can plot the different variables on the x or y axis.

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(16, 7))
hypso.plot(ax=ax1)
hypso.plot(x="water_level", y="volume", ax=ax2)
hypso.plot(x="area", y="volume", ax=ax3)



### properties and methods of the hypsometry
The hypsometry relationship has several properties that are useful to explore. Below we print examples of these. The comments describe what the different properties and methods do.

In [ ]:
# print the parameters of the fitted relationship
print(hypso.power_law_params)
# estimate some surface areas from water levels
wl = [1100, 1105]
areas = hypso.area_from_wl([1100, 1105])
print(f"area from water levels {wl}: {areas}")
# translate areas back to water levels
wl2 = hypso.wl_from_area(areas)
print(f"water levels from areas {areas}: {wl2}")
vol = hypso.volume_from_wl(wl)
# volume from the commensurate areas should be the same
vol2 = hypso.volume_from_area(areas)
print(f"Volumes computed from water levels: {vol}")
print(f"Volumes computed from areas (should be the same as from water levels): {vol2}")
      

### Store hypsometric points to a file
We can store the hypsometry relationship to a file for later use.

In [ ]:
hypso.to_file("static_hypso.csv")